***
10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke factor bt

get factor bt accuracy (+- std dev), wacc and tau

save in results/spammer_type/factor_bt.csv
***

## Device Setup

In [1]:
!nvidia-smi

Sun Dec 21 09:31:19 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   54C    P0              47W / 300W |      7MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


### Importing FactorBT

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from crowdkit.aggregation import NoisyBradleyTerry
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc
from pgem import EMWrapper

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Passage dataset

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/faceage/data/crowd_labels.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6306,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6176,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6175,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6174,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0


In [7]:
gt_df_path = "../../real_data/faceage/data/gt.csv"

In [8]:
import pandas as pd
gt_df = pd.read_csv(gt_df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
gt_df = sort_df(gt_df, 'label')
gt_df.head()

,label,score
4897,https://tlk.s3.yandex.net/annotation_tasks/IMD...,42
4901,https://tlk.s3.yandex.net/annotation_tasks/IMD...,42
5425,https://tlk.s3.yandex.net/annotation_tasks/IMD...,46
2054,https://tlk.s3.yandex.net/annotation_tasks/IMD...,23
7920,https://tlk.s3.yandex.net/annotation_tasks/IMD...,62


In [9]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [10]:
import pickle

with open("../../real_data/faceage/data/FaceAgeDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,full_path,score,gender
0,nm1442940_rm3965098752_1996-10-3_2006.jpg,10,0.0
1,nm4832920_rm1781768448_2003-8-28_2013.jpg,10,0.0
2,nm0652089_rm860657920_1992-3-10_2002.jpg,10,0.0
3,nm0004917_rm1493730304_1969-5-12_1979.jpg,10,0.0
4,nm1113550_rm1332711936_1996-4-14_2006.jpg,10,0.0
...,...,...,...
9145,475367_1941-08-03_2011.jpg,70,1.0
9146,304085_1919-07-07_1989.jpg,70,1.0
9147,nm0001627_rm4164078592_1927-2-20_1997.jpg,70,1.0
9148,nm0000024_rm1715129344_1904-4-14_1974.jpg,70,1.0


In [11]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

9150


In [12]:
def sort_df(df, column_name):
    # Sort by a specific column (replace 'column_name' with your column)
    df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

    return df_sorted

### Addition of Random Guessors

In [13]:
spammer_type = "random"

In [14]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [15]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [16]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

4500


### Addition of Anti-Personas

In [ ]:
spammer_type = "anti"

In [ ]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

### Addition of Left Position-Biased Spammers

In [ ]:
spammer_type = "left"

In [ ]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="left", seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

### Addition of Right Position Biased Spammers

In [ ]:
spammer_type = "right"

In [ ]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="right", seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")

### Addition of Equal Proportion of all four kind of spammers

In [ ]:
spammer_type = "equal"

In [ ]:
csv_file = f"results/{spammer_type}/factorbt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "FactorBT_acc_mean", "FactorBT_acc_std",
    "FactorBT_wacc_mean", "FactorBT_wacc_std",
    "FactorBT_tau_mean", "FactorBT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100
import pandas as pd

for percent in percents:
    # initialize metrics
    FactorBT_accs, FactorBT_waccs, FactorBT_taus = [], [], []
    
    for sd in range(20, 30):
        try:
            # get df
            random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
            K = len(list(random_df["performer"].unique()))
            print(K)
            random_df.rename(columns={"performer": "worker"}, inplace=True)
        
            agg_noisybt = NoisyBradleyTerry(n_iter=10).fit_predict(random_df)
            agg_noisybt_df = pd.DataFrame(list(agg_noisybt.items()), columns=['label', 'score'])
            factorbt_scores = list(agg_noisybt_df['score'])
            annot_bt_np = to_numpy(factorbt_scores)
            
            if np.isnan(annot_bt_np).any():
                    continue
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if FactorBT_tau < 0:
                annot_bt_np = -annot_bt_np
            FactorBT_acc = compute_acc(gt_df, annot_bt_np, device)
            FactorBT_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            FactorBT_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"FactorBT failed due to {e}")
            continue
        FactorBT_accs.append(FactorBT_acc)
        FactorBT_waccs.append(FactorBT_wacc)
        FactorBT_taus.append(FactorBT_tau)

    row = [
        percent,
        np.mean(FactorBT_accs), np.std(FactorBT_accs),
        np.mean(FactorBT_waccs), np.std(FactorBT_waccs),
        np.mean(FactorBT_taus), np.std(FactorBT_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"FactorBT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(FactorBT_accs):.4f} ± {np.std(FactorBT_accs):.4f} | "
    f"WAcc: {np.mean(FactorBT_waccs):.4f} ± {np.std(FactorBT_waccs):.4f} | "
    f"Tau: {np.mean(FactorBT_taus):.4f} ± {np.std(FactorBT_taus):.4f}")